For faster loading in main doc, and export
- Pre-crop listings file to be within Greater London Boundaries
- Reduce metrocrime file to Sep 2023
- Subset LSOA to only those in London Boundaries

In [5]:
# Load all libraries [To implement TRY EXCEPT]
import os
from requests import get
from urllib.parse import urlparse
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import glob 

In [ ]:
# Function to download files from remote
def cache_data(src:str, dest:str) -> str:
    """
    Create a folder to store file from URL.
    If folder doesn't already exists, then create one, before writing the file.

    src : URL
    dest : location on local drive
    
    """    
    url = urlparse(src) # We assume that this is some kind of valid URL 
    fn  = os.path.split(url.path)[-1] # Extract the filename
    dfn = os.path.join(dest,fn) # Destination filename as path
    
    if not os.path.isfile(dfn):
        
        print(f"{dfn} not found, downloading!")

        path = os.path.split(dest)
        
        if len(path) >= 1 and path[0] != '':
            os.makedirs(os.path.join(*path), exist_ok=True)
            
        with open(dfn, "wb") as file:
            response = get(src)
            file.write(response.content)
            
        print("\tDone downloading...")

    else:
        print(f"Found {dfn} locally!")

    return dfn

In [ ]:
# Define paths
ddir  = os.path.join('data') # destination directory
spath = 'https://github.com/jreades/i2p/blob/master/data/src/' # source path
spath_db = 'https://www.dropbox.com/scl/fi/' # source path for dropbox

# Read in airbnb, crime, and lsoa stats files. https://www.dropbox.com/home/casa/casa_fsds_cheers
# [PATIENCE] Takes about 4 minutes to download and read in the data

crime = pd.read_csv(cache_data(spath_db + 'tcnsqtzoodz3xz1m9gy8i/metrocrime2023.csv?rlkey=od64ji56onhaqobkit3vr5oih&dl=1', ddir),low_memory=False)
listings = pd.read_csv(cache_data(spath_db + '0v2xl5j4l2pvkxfolj5d0/london_listings_sep23.csv.gz?rlkey=feqrd33xwp8zdps2pqa9qzhao&dl=1', ddir),compression='gzip',low_memory=False)
lsoa = gpd.read_file(cache_data(spath_db + 'hda7eykjtjp760s1u4z05/LSOA_Dec_2021_Boundaries_Generalised.geojson?rlkey=9qj5zdxslgxpjshx41988oez1&dl=1', ddir))
water = gpd.read_file(cache_data(spath+'Water.gpkg?raw=true', ddir))
green = gpd.read_file(cache_data(spath+'Greenspace.gpkg?raw=true', ddir))
boros = gpd.read_file(cache_data(spath+'Boroughs.gpkg?raw=true', ddir))
print('Data loaded!')

In [ ]:
# Convert to geodataframes crime incidents, remove invalid geometries
crime = gpd.GeoDataFrame(crime, geometry=gpd.points_from_xy(crime['Longitude'], crime['Latitude']),crs='EPSG:4326')
crime = crime[~crime.geometry.is_empty] # Filtering out no geometry
crime = crime[crime['Month'].isin(['2023-09'])]  # To 95313 rows after filtering only Sep in 2023
print(f'The crime dataset has {crime.shape[0]} rows and {crime.shape[1]} columns.')

# Convert to geodataframes listins, remove invalid geometries
listings = gpd.GeoDataFrame(listings, geometry=gpd.points_from_xy(listings['longitude'], listings['latitude']), crs='EPSG:4326')
listings = listings[~listings.geometry.is_empty] # 87946 rows, no change
print(f'The listings dataset has {listings.shape[0]} rows and {listings.shape[1]} columns.')

In [ ]:
# Reproject everything to British National Grid
boros = boros.to_crs('epsg:27700')
lsoa = lsoa.to_crs('epsg:27700')
crime = crime.to_crs('epsg:27700')
listings = listings.to_crs('epsg:27700')

In [ ]:
# [Takes 3-4 minutes to run] Crop to London boundaries based on only London's LSOA from lsoa_stat   
listings = listings[listings.geometry.within(boros.geometry.unary_union)] # within boundaries
lsoa = lsoa[lsoa['LSOA21CD'].isin(lsoa_stat['LSOA code'].unique())] # LSOA in lsoa_stat (Greater-London-only)
crime = crime[crime['LSOA code'].isin(lsoa_stat['LSOA code'].unique())] # LSOA in lsoa_stat (Greater-London-only)

In [ ]:
# Export to geopackages
crime.to_file(os.path.join(ddir,'dropbox','metrocrime0923.gpkg'),driver='gpkg')
listings.to_file(os.path.join(ddir,'dropbox','london_listings0923.gpkg'), driver='gpkg')
lsoa.to_file(os.path.join(ddir,'dropbox','lsoa_full0923.gpkg'), driver='gpkg')

Concat excel files for LSOA stat into one df, and export

In [2]:
import pandas as pd 
import os 
import glob 
    
# use glob to get all the csv files  
# in the folder 
xlsx_files = glob.glob(os.path.join('data','lsoa_full',"*.xlsx")) 
  
dfs = []
# loop over the list of csv files 
for f in xlsx_files: 
    # read the  file 
    df = pd.read_excel(f,sheet_name='2021') 
    dfs.append(df)

dfs

[      LSOA code local authority code local authority name  All households   \
 0     E01000001            E09000001       City of London              838   
 1     E01000002            E09000001       City of London              824   
 2     E01000003            E09000001       City of London             1018   
 3     E01000005            E09000001       City of London              478   
 4     E01032739            E09000001       City of London              882   
 ...         ...                  ...                  ...              ...   
 4989  E01035718            E09000033          Westminster              732   
 4990  E01035719            E09000033          Westminster              634   
 4991  E01035720            E09000033          Westminster              587   
 4992  E01035721            E09000033          Westminster             1302   
 4993  E01035722            E09000033          Westminster             1178   
 
       Detached  Semi-detached  Terraced  Purpose 

In [3]:
dfss = [df.set_index('LSOA code') for df in dfs]
merged = pd.concat(dfss, axis=1).reset_index()
merged = merged.loc[:,~merged.columns.duplicated()].copy()

In [4]:
merged.to_csv(os.path.join('data','lsoa_full.csv'),index=False)